In [1]:
import numpy as np
import fenics as fe
from fenics import grad, dot, dx, ds
import matplotlib.pyplot as plt

In [2]:
n_nodes = 81
n_elements = n_nodes - 1
mesh = fe.UnitSquareMesh(n_elements, n_elements)
V = fe.FunctionSpace(mesh, 'Lagrange', 1)
mesh_spacing = 1 / n_elements
mesh_spacing

0.0125

In [3]:
# initial and Dirichlet boundary condition
ic_expression = fe.Constant(0)
u0 = fe.interpolate(ic_expression, V)
u1 = fe.interpolate(ic_expression, V)

def boundary(x, on_boundary):
    return on_boundary and fe.near(x[1], 0, 1e-14)

omega = 50 # Hz
bc_expression = fe.Expression(f'sin(2 * 3.1415 * omega * t)', degree=2, t=0, omega=omega)
bc = fe.DirichletBC(V, bc_expression, boundary)

In [4]:
# source term and Neumann boundary condition
f = fe.Expression('0', degree=0)
g = fe.Expression('0', degree=1)

In [5]:
# wave speed and time stepping parameters

# specify wave speed in meters per second (from physical constants)
rho = 1000 # kg / m^3
mu  = 4000 # Pa = N / m^2 = (kg m/s^2) / m^2 = kg / (m s^2)
c = np.sqrt(mu / rho) # (kg / (m s^2)) / (kg / m^3) = m^3 / (m s^2) = (m / s)^2

#c = 2 # meters / second

# dimensionalize the wave speed
L = 0.2 # side length in meters
c /= L # side lengths per second
print(c)

dt = 1e-4 # seconds

# check CFL condition
cfl_number = c * dt / mesh_spacing
print(cfl_number)
assert cfl_number <= 1, f'CFL condition violated ({c} x {dt} / {mesh_spacing} <= 1)'

total_time = 0.4 # seconds
n_steps = np.ceil(total_time / dt).astype(int)
n_steps

10.0
0.08


4000

In [6]:
# variational form
u = fe.TrialFunction(V)
v = fe.TestFunction(V)
a = (
    u * v * dx + 
    (c * dt)**2 * dot(grad(u), grad(v)) * dx
)
L = (
    (2 * u1 - u0 + dt**2 * f) * v * dx + 
    (g * v) * ds
)

In [25]:
# damped version

class AbsorbingBoundary(fe.UserExpression):
    def value_shape(self):
        return ()
    def eval(self, values, x):
        if x[0] < 0.05 or x[0] > 0.95 or x[1] > 0.95:
            values[0] = -1000
        else:
            values[0] = -10

k = fe.interpolate(AbsorbingBoundary(), V)

u = fe.TrialFunction(V)
v = fe.TestFunction(V)
a = (
    (1 - k * dt) * u * v * dx + 
    (c * dt)**2 * dot(grad(u), grad(v)) * dx
)
L = (
    ((2 - k * dt) * u1 - u0 + dt**2 * f) * v * dx + 
    (g * v) * ds
)

In [26]:
# get mapping from dofs to array indices
xyz = V.tabulate_dof_coordinates()
x = xyz[:,0]
y = xyz[:,1]
print(xyz.shape)

inds = np.argsort([n_nodes * x + y for x,y in xyz])
xyz[inds]

(6561, 2)


array([[0.    , 0.    ],
       [0.    , 0.0125],
       [0.    , 0.025 ],
       ...,
       [1.    , 0.975 ],
       [1.    , 0.9875],
       [1.    , 1.    ]])

In [27]:
u0.interpolate(ic_expression)
u1.interpolate(ic_expression)

u = fe.Function(V)
array = np.zeros((n_steps + 1, n_nodes, n_nodes))

t = 0
for i in range(n_steps):
    t += dt
    bc_expression.t = t
    fe.solve(a == L, u, [bc])
    u0.assign(u1)
    u1.assign(u)

    values = u.vector().get_local()
    array[i+1,...] = values[inds].reshape((n_nodes, n_nodes))

    if i % 10 == 0:
        print(f'Iteration {i+1} / {n_steps}')

Solving linear variational problem.
Iteration 1 / 4000
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Iteration 11 / 4000
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Iteration 21 / 4000
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solvi

In [28]:
import xarray as xr
import hvplot.xarray

array = xr.DataArray(
    array,
    dims=['t', 'x', 'y'],
    coords={
        't': np.arange(0, n_steps + 1) * dt,
        'x': np.linspace(0, 1, n_nodes),
        'y': np.linspace(0, 1, n_nodes)
    }
)
array.hvplot(groupby=['t'], x='x', y='y', data_aspect=1, clim=(-2, 2), cmap='seismic', widget_type='scrubber')

Row
    [0] HoloViews(DynamicMap, sizing_mode='fixed', widget_type='scrubber')
    [1] WidgetBox(align=('end', 'start'))
        [0] Player(end=4000, width=550)

In [29]:
array.hvplot(groupby=['t', 'x'], x='y', ylim=[-2, 2], widget_location='bottom')

Column
    [0] HoloViews(DynamicMap, height=300, sizing_mode='fixed', widget_location='bottom', width=700)
    [1] WidgetBox(align=('center', 'end'))
        [0] DiscreteSlider(margin=(20, 20, 5, 20), name='t', options=OrderedDict([('0', ...]), value=0.0, width=250)
        [1] DiscreteSlider(margin=(5, 20, 20, 20), name='x', options=OrderedDict([('0', ...]), value=0.0, width=250)